In [ ]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); 
using ForwardDiff
using Test
using RobotZoo
using RobotDynamics
using LinearAlgebra
using StaticArrays
using SparseArrays
using Printf
using MeshCat
using Plots

include("quadratic_cost.jl")
include("../test/nlp_test.jl")
include("cartpole.jl")

# Q1: Sequential Quadratic Programming (SQP) (50 pts)
In this problem you'll solve the canonical cartpole swing-up problem using sequential-quadratic programming (SQP), as we learned in class. While coding up a complete SQP solver is very challenging, we'll simplify it by only considering problems of the form:

$$\begin{aligned} &\text{minimize}_z && \frac{1}{2} z^T Q z + q^T z \\
&\text{subject to} && c(z) = 0 \end{aligned} $$

I.e. we'll only have you worry about equality constraints. Your overall algorithm should run something like this:
1. Initialization
2. Build the QP for the current step by taking Taylor series approximations of the cost and constraints
3. Solve the QP to get search directions for you primal and dual variables
4. Use a line search (or other globalization strategy) to find a good step
5. If termination conditions are satisfied, exit; otherwise, go to 2

## The Problem
You likely have already seen the cartpole swing-up problem previously. As shown in the picture below, the system is comprised of a pendulum attached to a cart, where forces can only be applied to the cart. The goal is to balance the pendulum above the cart. The system dynamics can be written as:

$$ x = \begin{bmatrix} y \\ \theta \\ v \\ \omega \end{bmatrix}, \quad \dot{x} = \begin{bmatrix} \dot{q} \\ \ddot{q} \end{bmatrix}, \quad
q = \begin{bmatrix} y \\ \theta \end{bmatrix}, \quad
\ddot{q} = -H^{-1} (C \dot{q} + G - B u)$$

where 
$$ H = \begin{bmatrix} m_c + m_p & m_p l \cos{\theta} \\
m_p l \cos{\theta} & m_p l^2 \end{bmatrix}, \;
C = \begin{bmatrix} 0 & -m_p \omega l \sin{\theta} \\ 0 & 0 \end{bmatrix}, \;
G = \begin{bmatrix} 0 \\ m_p g l \sin{\theta} \end{bmatrix}, \;
B = \begin{bmatrix} 1 \\ 0 \end{bmatrix} $$

with the following parameters:
* $m_p$: mass of the pole
* $m_c$: mass of the cart
* $g$: gravity
* $l$: length of the rod

In [ ]:
# Build the problem
model = RobotZoo.Cartpole()
n,m = size(model)
T = 101
tf = 2.0
dt = tf / (T-1)

# Initial & final condition
x0 = @SVector zeros(n)
xf = SA[0,pi,0,0];

# Cost function
Q = Diagonal(fill(1e-2,n))
R = Diagonal(fill(1e-1,m))
Qf = Diagonal(fill(1e1,n))
costfun = LQRCost(Q,R,xf)
costterm = LQRCost(Qf,R,xf)
obj = push!(fill(costfun,T-1), costterm)

# Initial Guess (linear interpolation)
X = [x0 + (xf - x0)*t for t in range(0,1, length=T)]
U = [@SVector zeros(m) for k = 1:T-1];

## Part (a): Evaluate the NLP (14 pts)
Before we put together our SQP method, let's write some convenient methods for evaluating the information we'll need from the NLP. We've given you a struct `NLP` below that contains all the information you'll need to evaluate things like the cost function, constraints, and their derivatives.

In [ ]:
"""
    NLP{n,m,L,Q}

Represents a (N)on(L)inear (P)rogram of a trajectory optimization problem,
with a dynamics model of type `L`, a quadratic cost function, horizon `T`, 
and initial and final state `x0`, `xf`.

The kth state and control can be extracted from the concatenated state vector `Z` using
`Z[nlp.xinds[k]]`, and `Z[nlp.uinds[k]]`.

# Constructor
    NLP(model, obj, tf, T, x0, xf, [integration])

# Basic Methods
    Base.size(nlp)    # returns (n,m,T)
    num_ineq(nlp)     # number of inequality constraints
    num_eq(nlp)       # number of equality constraints
    num_primals(nlp)  # number of primal variables
    num_duals(nlp)    # total number of dual variables
    packZ(nlp, X, U)  # Stacks state `X` and controls `U` into one vector `Z`

# Evaluating the NLP
The NLP supports the following API for evaluating various pieces of the NLP:

    eval_f(nlp, Z)         # evaluate the objective
    grad_f!(nlp, grad, Z)  # gradient of the objective
    hess_f!(nlp, hess, Z)  # Hessian of the objective
    eval_c!(nlp, c, Z)     # evaluate the constraints
    jac_c!(nlp, c, Z)      # constraint Jacobian
"""
struct NLP{n,m,L,Q}
    model::L                                 # dynamics model
    obj::Vector{QuadraticCost{n,m,Float64}}  # objective function
    T::Int                                   # number of knot points
    tf::Float64                              # total time (sec)
    x0::MVector{n,Float64}                   # initial condition
    xf::MVector{n,Float64}                   # final condition
    xinds::Vector{SVector{n,Int}}            # Z[xinds[k]] gives states for time step k
    uinds::Vector{SVector{m,Int}}            # Z[uinds[k]] gives controls for time step k
    times::Vector{Float64}                   # vector of times
    function NLP(model::AbstractModel, obj::Vector{<:QuadraticCost{n,m}},
            tf::Real, T::Integer, x0::AbstractVector, xf::AbstractVector, integration::Type{<:QuadratureRule}=RK4
        ) where {n,m}
        xinds = [SVector{n}((k-1)*(n+m) .+ (1:n)) for k = 1:T]
        uinds = [SVector{m}((k-1)*(n+m) .+ (n+1:n+m)) for k = 1:T-1]
        times = collect(range(0, tf, length=T))
        new{n,m,typeof(model), integration}(
            model, obj,
            T, tf, x0, xf, xinds, uinds, times
        )
    end
end
Base.size(nlp::NLP{n,m}) where {n,m} = (n,m,nlp.T)
num_primals(nlp::NLP{n,m}) where {n,m} = n*nlp.T + m*(nlp.T-1)
num_duals(nlp::NLP) = num_eq(nlp) + num_ineq(nlp)
num_eq(nlp::NLP{n,m}) where {n,m} = n*nlp.T + n
num_ineq(nlp::NLP) = 0

"""
    packZ(nlp, X, U)

Take a vector state vectors `X` and controls `U` and stack them into a single vector Z.
"""
function packZ(nlp, X, U)
    Z = zeros(num_primals(nlp))
    for k = 1:nlp.T-1
        Z[nlp.xinds[k]] = X[k]
        Z[nlp.uinds[k]] = U[k]
    end
    Z[nlp.xinds[end]] = X[end]
    return Z
end

"""
    unpackZ(nlp, Z)

Take a vector of all the states and controls and return a vector of state vectors `X` and
controls `U`.
"""
function unpackZ(nlp, Z)
    X = [Z[xi] for xi in nlp.xinds]
    U = [Z[ui] for ui in nlp.uinds]
    return X, U
end

### Useful Examples
You may find the following code snippets helpful as you complete the methods for the NLP.

In [ ]:
# Create NLP
nlp = NLP(model, obj, tf, T, x0, xf)

# Create a vector of all states and controls
Z = packZ(nlp, X, U)

# Unpack into states and vectors
X2, U2 = unpackZ(nlp, Z)

# Evaluate the cost
cost(nlp.obj, X, U)

# Get kth state, control
k = 10
x = Z[nlp.xinds[k]]
u = Z[nlp.uinds[k]]

# Dynamics
t = nlp.times[k]
dt = nlp.times[k+1] - nlp.times[k[]]
discrete_dynamics(RK4, nlp.model, x, u, t, dt)

# Dynamics Jacobian
∇f = zeros(n,n+m)
discrete_jacobian!(RK4, ∇f, nlp.model, x, u, t, dt)

# 2nd-order Dynamics derivative
#   derivative of ∇f'b
∇jac = zeros(n+m,n+m)
b = zeros(n)
∇discrete_jacobian!(RK4, ∇jac, nlp.model, x, u, t, dt, b)

### Objective
Complete the following methods to evaluate the objective and its derivatives.

In [ ]:
# TASK: Finish the following methods
#       eval_f  (2 pts)
#       grad_f! (2 pts)
#       hess_f! (2 pts)

"""
    eval_f(nlp, Z)

Evaluate the objective, returning a scalar.
"""
function eval_f(nlp::NLP, Z)
    # TODO: compute the objective value (cost)
    J = 0.0
    return J
end

"""
    grad_f!(nlp, grad, Z)

Evaluate the gradient of the objective at `Z`, storing the result in `grad`.
"""
function grad_f!(nlp::NLP{n,m}, grad, Z) where {n,m}
    xi,ui = nlp.xinds, nlp.uinds
    obj = nlp.obj
    for k = 1:nlp.T-1
        x,u = Z[xi[k]], Z[ui[k]]
        # TODO: Compute the cost gradient
        grad[xi[k]] .= 0
        grad[ui[k]] .= 0
    end
    grad[xi[end]] = obj[end].Q*Z[xi[end]] + obj[end].q
    return nothing
end

"""
    hess_f!(nlp, hess, Z)

Evaluate the Hessian of the objective at `Z`, storing the result in `hess`.
Should work with `hess` sparse.
"""
function hess_f!(nlp::NLP{n,m}, hess, Z, rezero=true) where {n,m}
    # TODO: Compute the objective hessian
end

## Constraints
Complete the following methods to evaluate the constraints and their derivatives.

In [ ]:
# TASK: Complete the following methods
#       eval_c!  (2 pts)
#       jac_c!   (2 pts)
#       ∇jvp!    (2 pts)
"""
    eval_c!(nlp, c, Z)

Evaluate the equality constraints at `Z`, storing the result in `c`.
The constraints should be ordered as follows: 
1. Initial condition ``x_1 = x_\\text{init}``
2. Dynamics ``f(x_k,u_k) - x_{k+1} = 0``
3. Terminal constraint ``x_T = x_\\text{goal}``
"""
function eval_c!(nlp::NLP{n,m,<:Any,Q}, c, Z) where {n,m,Q}
    T = nlp.T
    xi,ui = nlp.xinds, nlp.uinds
    idx = xi[1]

    # TODO: initial condition

    # dynamics
    for k = 1:T-1
        idx = idx .+ n
        x,u = Z[xi[k]], Z[ui[k]]
        x⁺ = Z[xi[k+1]]
        dt = nlp.times[k+1] - nlp.times[k]
        
        # TODO: Dynamics constraint
        c[idx] .= 0
        
    end

    # TODO: terminal constraint
    idx = idx .+ n
    c[idx] .= 0
    return nothing
end

"""
    jac_c!(nlp, jac, Z)

Evaluate the constraint Jacobian, storing the result in the (potentially sparse) matrix `jac`.
"""
function jac_c!(nlp::NLP{n,m,<:Any,Q}, jac, Z) where {n,m,Q}
    # TODO: Initial condition

    xi,ui = nlp.xinds, nlp.uinds
    idx = xi[1]
    for k = 1:nlp.T-1
        idx = idx .+ n 
        zi = [xi[k];ui[k]]
        zi2 = k < T-1 ? zi .+ (n+m) : xi[T]
        x = Z[xi[k]]
        u = Z[ui[k]]
        t = nlp.times[k]
        dt = nlp.times[k+1] - nlp.times[k]

        ∇f = view(jac, idx, zi)
        ∇f2 = view(jac, idx, zi2)
        
        # TODO: Dynamics constraint
    end
    idx = idx .+ n 
    
    # TODO: Terminal constraint
end

## Lagrangian

In [ ]:
# TASK: Complete the following methods
#       lagrangian         (2 pts)
#       grad_lagrangian!   (2 pts)

"""
    lagrangian(nlp, Z, λ, c)

Evaluate the Lagrangian at `Z` and `λ`. Calculates the constraints, storing the result in `c`.

The sign on the multipliers should be negative, e.g. J(x) - λ'c(x)
"""
function lagrangian(nlp::NLP{n,m}, Z, λ, c=zeros(eltype(Z),length(λ))) where {n,m}
    # TODO: Calculate the Lagrangian
    L = 0
    return L
end

"""
    grad_lagrangian(nlp, grad, Z, λ)

Evaluate the gradient of the Lagrangian.
"""
function grad_lagrangian!(nlp::NLP{n,m}, grad, Z, λ, jac = spzeros(length(λ), length(Z))) where {n,m}
    # TODO: Calculate the gradient of the Lagrangian, store the result in grad
    return nothing
end

## Residuals
Feel free to use these function to evaluate the primal and dual residuals.

In [ ]:
"""
    primal_residual(nlp, Z, λ, [g; p])

Evaluate the `p`-norm of the primal residual (stationarity condition).
"""
function primal_residual(nlp::NLP, Z, λ, g=zeros(num_primals(nlp)); p=2)
    grad_lagrangian!(nlp, g, Z, λ)
    return norm(g, p)
end

"""
    dual_residual(nlp, Z, λ, [c; p])

Evaluate the `p`-norm of the dual residual (constraint violation).
"""
function dual_residual(nlp::NLP, Z, λ, c=zeros(num_eq(nlp)); p=2)
    eval_c!(nlp, c, Z)
    norm(c, p)
end

### Testing the Methods

In [ ]:
nlp = NLP(model, obj, tf, T, x0, xf)
Z = packZ(nlp, X, U)
λ = zeros(num_duals(nlp));

In [ ]:
include("../test/nlp_test.jl")

In [ ]:
# Test methods
test_nlp(nlp, Z, λ);

## Part (b): Solving the QP (8 pts)
Now that we have methods to evaluate our NLP and its derivatives, we need some methods to form the QP from our NLP and then solve it. Use the struct we've provided below and implement the `build_qp!` method to update the QP sub-problem from the current primal and dual variables, as well as the `solve_qp!` method to solve it for the step directions in the primal and dual variables. Note that since our NLP only has equality constraints, the QP sub-problem will also only have equality constraints, so it can be solved directly using a linear solve on the KKT conditions.

In [ ]:
@doc raw"""
    TOQP

A type specifying a (T)rajectory (O)ptimization (Q)uadratic (P)rogram, of the form


``\begin{aligned} &\text{minimize} &&\frac{1}{2} z^T Q z + q^T z \\ 
&\text{subject to} && A z = b \\ 
&&& l \leq C z \leq u \end{aligned}``

where ``z = [x_1^T \; u_1^T \; \dots \; x_{T-1}^T \; u_{T-1}^T \; x_T^T]^T`` and 
``x \in \mathbb{R}^n`` is the state vector and ``u \in \mathbb{R}^m`` is the control vector.

# Constructors

    TOQP(n,m,T,M,P)

where `n` is the number of states, `m` is the number of controls, `T` is the horizon, `M` is the number of equality 
constraints, and `P` is the number of inequality constraints.

# Methods

    num_ineq(qp)     # number of inequality constraints
    num_eq(qp)       # number of equality constraints
    num_primals(qp)  # number of primal variables
    num_duals(qp)    # total number of dual variables


"""
struct TOQP
    Q::SparseMatrixCSC{Float64,Int}  # quadratic cost
    q::Vector{Float64}               # linear cost
    A::SparseMatrixCSC{Float64,Int}  # equality constraint Ax = b
    b::Vector{Float64}               # equality constraint 
    C::SparseMatrixCSC{Float64,Int}  # inequality constraint l ≤ Cx ≤ u
    l::Vector{Float64}               # inequality constraint lower bound
    u::Vector{Float64}               # inequality constraint upper bound
    n::Int
    m::Int
    T::Int

    function TOQP(n,m,T,M,P)
        N = n*T + (T-1)*m
        Q = spzeros(N,N)
        q = zeros(N)
        A = spzeros(M,N)
        b = zeros(M) 
        C = spzeros(P,N)
        l = fill(-Inf,P)
        u = fill(Inf,P)

        new(Q,q,A,b,C,l,u,n,m,T)
    end
end


function TOQP(nlp::NLP{n,m}) where {n,m}
    TOQP(n,m,nlp.T, num_eq(nlp), num_ineq(nlp))
end

num_ineq(qp::TOQP) = length(qp.l)
num_eq(qp::TOQP) = length(qp.b)
num_primals(qp::TOQP) = length(qp.q)
num_duals(qp::TOQP) = num_ineq(qp) + num_eq(qp)

### Build the QP

In [ ]:
# TASK: Complete the following method to build the QP sub-problem (3 pts)
"""
    build_qp!(qp, nlp, Z, λ; [gn=true])

Build a QP from the NLP, evaluated at primal variables `Z` and dual variables `λ`, 
optionally using either the Hessian of the cost function (`gn = true`) or the Hessian of the Lagrangian (`gn = false`).
"""
function build_qp!(qp::TOQP, nlp::NLP, Z, λ; gn::Bool=true)
    # TODO: Build the qp, filling in qp.Q, qp.q, qp.A, qp.b
    return nothing
end

### Solve the QP

In [ ]:
# TASK: Complete the function to solve the QP (5 pts)
"""
    solve_qp!(qp, [reg])

Solve the QP, optionally applying regularization `reg`.
"""
function solve_qp!(qp::TOQP, reg=0.0)
    # TODO: Solve the QP sub-problem
    # HINT: Form the KKT system and solve with a single linear solve
    N,M = num_primals(qp), num_duals(qp)
    dZ = zeros(N)
    dλ = zeros(M)
    return dZ, dλ
end

## Part (c): Writing the SQP Method (15 pts)
Now that you have all the pieces, put them all together into an SQP method. We provide some more details about the line search we reccommend you implement below.

### Line Search
For SQP, we reccommend using the "exact" $\ell_1$ merit function:

$$ \phi_1(x; \mu) = f(x) + \mu ||c(x)||_1 $$

whose directional derivative in direction $p_k$ can be computed using:

$$ D\left(\phi_1(x_k; \mu); p_k\right) = \nabla f_k^T p_k - \mu ||c_k||_1 $$

The penalty parameter $\mu$ should always have a value greater than or equal to this expression:

$$ \mu \geq \frac{\nabla f_k^T p_k + (\sigma/2) p_k^T \nabla_{xx}^2 \mathcal{L}_k p_k}{(1-\rho) ||c_k||_1} $$

where $\rho = 0.5$, and

$$ \sigma = \begin{cases} 1 & \text{if } p_k^T \nabla_{xx}^2 \mathcal{L}_k p_k > 0 \\
0 & \text{otherwise} \end{cases} $$.

Here $p_k$ is the step direction, $\nabla f_k$ is the gradient of the cost function at the current value of $x$, $c_k$ are the constraint values at the current value of $x$, and $\nabla_{xx}^2 \mathcal{L}_k$ is the Hessian of the Lagrangian for the current value of $x$. For more information, see Nocedal and Wright Ch 18.3

In [49]:
# TASK: Complete the SQP method (15 pts)
"""
    solve_sqp(nlp, Z, λ; kwargs...)

Solve the trajectory optimization problem specified by `nlp` using Sequential Quadratic Programming, given the initial 
guess for the primal variables `Z` and `λ`.
"""
function solve_sqp(nlp, Z0, λ0;
        iters=100,                   # max number of iterations
        verbose=0,                   # verbosity level
        eps_primal=1e-6,             # primal feasibility tolerance
        eps_dual=1e-4,               # dual feasibility tolerance
        eps_fn=sqrt(eps_primal),     # 
        gn::Bool=true,               # use Gauss-Newton approximation
        enable_soc::Bool=true,       # enable Second-Order-Corrections during the line search
        ls_iters=10,                 # max number of line search iterations
        reg_min=1e-6,                # minimum regularization
    )
    t_start = time_ns()

    # Initialize solution
    Z = deepcopy(Z0)
    λ = deepcopy(λ0)
    qp = TOQP(nlp)

    # Line Search parameters
    μ = 10.0                         # initial merit function penalty
    reg = reg_min                    # regularization

    # Stats
    # TODO: Fill this is during your solve
    stats = Dict(
        :cost => Float64[],
        :viol_primal => Float64[],  # constraint violation
        :viol_dual => Float64[],    # stationarity
        :time => Float64[]
    )

    # Primary Loop
    for iter = 1:iters
        #### TODO: Check Convergence ####

        #### TODO: Build and Solve QP ####
        
        
        #### TODO: Penalty Update ####
        

        #### TODO: Line Search Initialization ####
        
        # Params
        α = 1.0                                # line search parameter
        J0 = eval_f(nlp, Z)                    # initial cost
        phi0 = Inf                             # initial merit function value
        dphi0 = Inf                            # initial merit function gradient

        #### TODO: Backtracking line search ####
        
        
        #### TODO: Apply Step ####

        # Output
        verbose > 0 && @printf("   α = %0.2f, ΔJ: %0.2e, Δϕ: %0.2e, reg: %0.2e, pen: %d\n", 
            α, J - eval_f(nlp, Z), phi0 - phi, reg, μ)
        push!(stats[:time], (time_ns() - t_start) / 1e6)  # ms
    end
    
    return Z, λ, stats 
end

solve_sqp

In [ ]:
# TASK: Solve the problem with SQP
#       Save the output to Zgn, λgn, stats_gn
#       Solve to a tolerance of 1e-6 for constraint satisfaction and 1e-4 for stationarity condition

Z = packZ(nlp, X, U)
λ = zeros(num_duals(nlp))
Zgn, λgn, stats_gn = solve_sqp(nlp, Z, λ, verbose=0, gn=true);
length(stats_gn[:time])

## Visualization

In [ ]:
vis = Visualizer()
set_mesh!(vis, model)
render(vis)

In [ ]:
Xgn = [Zgn[xi] for xi in nlp.xinds]
visualize!(vis, model, tf, Xgn)

## Part (d): Implement Full Newton (4 pts)
To compute the full Newton step, we need to evaluate the second-order derivatives of our constraints. Complete the methods below to calculate the full 2nd-order expansion of the Lagrangian.

In [ ]:
# TASK: Finish the methods below
#       ∇jvp!              (2 pts)
#       hess_lagrangian!   (2 pts)

"""
    ∇jvp!(nlp, hess, Z, λ)

Evaluate the Jacobian of the constraint Jacobian-transpose vector product, e.g. ``\\frac{\\partial}{\\partial z} \\nabla c^T \\lambda``,
storing the result in the (potentially sparse) matrix `hess`.
"""
function ∇jvp!(nlp::NLP{n,m,<:Any,Q}, hess, Z, λ) where {n,m,Q}
    xi,ui = nlp.xinds, nlp.uinds
    idx = [xi[1]; ui[1]]
    idx2 = xi[1]
    
    # TODO: Initial Constraint
    
    # Dynamics constraints
    for k = 1:nlp.T-1
        idx2 = idx2 .+ n
        zi2 = k < T-1 ? idx .+ (n+m) : xi[T]
        x = Z[xi[k]]
        u = Z[ui[k]]
        λk = λ[idx2]
        t = nlp.times[k]
        dt = nlp.times[k+1] - nlp.times[k]
        
        ∇f = view(hess, idx, idx)
        ∇f2 = view(hess, idx, zi2)
        
        # TODO: Calculate second derivative the dynamics
        
        # Advance indices
        idx = idx .+ (n + m)
    end
    # TODO: Terminal constraint
end

"""
    hess_lagrangian(nlp, grad, Z, λ)

Evaluate the Hessian of the Lagrangian.
"""
function hess_lagrangian!(nlp::NLP{n,m}, hess, Z, λ) where {n,m}
    # TODO: Calculate the Hessian of the Lagrangian, store the result in hess
    return nothing
end


In [ ]:
# Test the methods
test_nlp(nlp, Z, λ, full_newton=true);

## Part (e): Solve with Full Newton (2 pts)
Now that we can calculate our full Newton step, modify `build_qp!` to use `hess_lagrangian!` when the `gn` is false. Then solve the problem with the full Newton solve.

In [ ]:
# TASK: Do the following
#       1. Modify build_qp! to use the full Newton Hessian when `gn = false`
#       2. Solve the problem using full Newton steps. Save the result in Zfn, λfn, stats_fn

Z = packZ(nlp, X, U)
λ = zeros(num_duals(nlp))
Zfn, λfn, stats_fn = solve_sqp(nlp, Z, λ, verbose=0, gn=false, eps_dual=1e-4);
length(stats_fn[:time])

## Part (f): Convergence Comparison (2 pts)
Let's compare the convergence of Gauss Newton versus Full Newton. Generate the following plots:
1. Constraint violation vs iterations. Include a line for both Gauss and Full Newton
2. Constraint violation vs time. Include a line for both Gauss and Full Newton

Once you've generated the plots, take a minute to think about they tell you. What are the tradeoffs of using Gauss-Newton vs full Newton?

You are free to use whatever plotting library you want (we suggest PyPlot.jl or Plots.jl).

**TIP**: Save the data you need in the stats dictionary that returned from the sqp solver.

**TIP**: If your plot vs time has a large offset before the first iteration, run your solver again (remember it has to compile the first time through).

In [ ]:
# TASK: Plot constraint violation vs iterations

In [ ]:
# TASK: Plot constraint violation vs time

## Part (g): Track the solution with TVLQR (5 pts)
Let's now use our trajectory and simulate it on a system with some model mismatch.

**TASK**:
1. Generate a TVLQR controller that tracks your optimized trajectories.
2. Run your controller on a simulated cartpole with a cart mass of 1.1 kg instead of 1 kg. Get it to successfully stabilize. The final stabilized position doesn't have to to be at an x-position of 0. Simulate for at least 4 seconds.

**TIPS**:
1. Use code you've generated previously to build your TVLQR controller
2. If your cartpole gets it to the top but doesn't stabilize it for the full 4 seconds, think about how you could design your controller to stabilize it...

In [ ]:
# TASK: Generate a TVLQR controller
#       Store your controller as the variable `ctrl`

# TODO: Replace this with a controller of your choice
ctrl = NullController(model)

# SOLUTION
using ControlSystems

function tvlqr(A,B,Q,R,Qf)
    # Extract some variables
    T = length(A)+1
    n,m = size(B[1])
    P = [zeros(n,n) for k = 1:T]
    K = [zeros(m,n) for k = 1:T-1]
    
    P[end] .= Qf
    for k = reverse(1:T-1) 
        K[k] .= (R + B[k]'P[k+1]*B[k])\(B[k]'P[k+1]*A[k])
        P[k] .= Q + A[k]'P[k+1]*A[k] - A[k]'P[k+1]*B[k]*K[k]
    end
    
    return K,P
end

"""
    LQRController

A TVLQR controller that tracks the trajectory specified by `Xref` and `Uref`
using the linear feedback gains `K`.
"""
struct LQRController
    K::Vector{Matrix{Float64}}
    Xref::Vector{Vector{Float64}}
    Uref::Vector{Vector{Float64}}
    times::Vector{Float64}
end
get_k(controller, t) = searchsortedlast(controller.times, t)

function get_control(ctrl::LQRController, x, t)
    k = get_k(ctrl, t)
    K = ctrl.K[k]
    return ctrl.Uref[k] - K*(x - ctrl.Xref[k])
end

# Generate A,B matrices
Xfn,Ufn = unpackZ(nlp, Zfn)
A = [zeros(n,n) for k = 1:T-1]
B = [zeros(n,m) for k = 1:T-1]
∇f = RobotDynamics.DynamicsJacobian(model)
for k = 1:T-1
    local t = nlp.times[k]
    local dt = nlp.times[k+1] - nlp.times[k]
    discrete_jacobian!(RK4, ∇f, model, Xfn[k], Ufn[k], t, dt)
    A[k] .= ∇f.A
    B[k] .= ∇f.B
end


Kinf = dlqr(A[end], B[end], Matrix(Q), Matrix(R))

K, = tvlqr(A,B,Q,R,Qf)
push!(K,Kinf)
push!(Ufn,zeros(m))
ctrl = LQRController(K, Xfn, Ufn, nlp.times);

In [ ]:
render(vis)

In [ ]:
# Simulate with a different model
model2 = RobotZoo.Cartpole(1.1, 0.2, 0.5, 9.81)
Xsim, Usim, tsim = simulate(model2, x0, ctrl, tf=2tf)
visualize!(vis, model, tsim[end], Xsim)

## Tests

In [ ]:
run_tests();